Generating transition list for skyline for a given molecule ID

In [4]:
# Generate a Skyline transition list **for a single row number** (e.g., 10)
# Ignores all other columns in the file.
#
# Expected columns in the CSV (case-insensitive; other columns are ignored):
#   - molecule
#   - matched_z_list
#   - matched_mz_list
#   - (optional) polarity / plolarity
#
# Output CSV columns (Skyline Small Molecule transition list):
#   Molecule, Precursor m/z, Precursor Adduct, Polarity
#
# Notes:
# - ROW_NUM is 1-based (ROW_NUM=10 means "the 10th row" in the file).
# - Charges of 0 are discarded. If polarity is missing/blank, defaults to Positive.

import re
from pathlib import Path
from typing import List, Optional
import pandas as pd

# ----------------------------
# EDIT THESE IF NEEDED
# ----------------------------
INPUT_PATH  = r"F:/input.csv"               # or "input.csv" or absolute path
OUTPUT_PATH = r"F:/transition_list_row0.csv"
ROW_NUM     = 1  # 1-based: e.g., 10 -> process only the 10th row
# ----------------------------

def extract_int_list(s) -> List[int]:
    if pd.isna(s):
        return []
    # Accept anything like: [2, 3, 4], "2,3,4", "z=2;3;4", etc.
    return [int(x) for x in re.findall(r"[-+]?\d+", str(s))]

def extract_float_list(s) -> List[float]:
    if pd.isna(s):
        return []
    # Accept 123, 123.4, .45, 1e3, -2.5E-2, etc.
    pattern = r"[-+]?(?:\d+\.\d*|\.\d+|\d+)(?:[eE][-+]?\d+)?"
    return [float(x) for x in re.findall(pattern, str(s))]

def norm_polarity(val) -> str:
    if val is None or str(val).strip().lower() in ("", "nan", "none"):
        return "Positive"
    return "Negative" if str(val).strip().lower().startswith("neg") else "Positive"

def adduct_from_charge(z: int, polarity: str = "Positive") -> str:
    # Skyline adduct notation from integer charge + polarity
    try:
        z = int(round(float(z)))
    except Exception:
        z = 1
    if z == 0:
        z = 1
    if str(polarity).lower().startswith("neg"):
        return f"[M-{abs(z)}H]{abs(z)}-"
    else:
        return f"[M+{abs(z)}H]{abs(z)}+"

def find_col(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    # Case-insensitive lookup; returns the actual column name if found
    lower_map = {c.lower(): c for c in df.columns}
    for cand in candidates:
        c = cand.lower()
        if c in lower_map:
            return lower_map[c]
    return None

# --- Load CSV (ignore extra columns, we will only touch what we need) ---
p = Path(INPUT_PATH)
if not p.exists():
    raise FileNotFoundError(f"Input file not found: {INPUT_PATH}")

# Read all (cheapest is to read the file; we will ignore extra columns downstream)
df_full = pd.read_csv(p, low_memory=False)

# Validate row number (1-based)
if ROW_NUM < 1 or ROW_NUM > len(df_full):
    raise IndexError(f"ROW_NUM={ROW_NUM} is out of range (file has {len(df_full)} rows).")

# Pick only the single row requested
df = df_full.iloc[[ROW_NUM - 1]].copy()

# --- Identify required columns (case-insensitive) ---
molecule_col = find_col(df, ["molecule"])
zlist_col    = find_col(df, ["matched_z_list"])
mzlist_col   = find_col(df, ["matched_mz_list"])
pol_col      = find_col(df, ["polarity", "plolarity"])  # accept common misspelling

if not molecule_col or not zlist_col or not mzlist_col:
    raise ValueError(
        "Missing required columns. Need at least: 'molecule', 'matched_z_list', 'matched_mz_list'. "
        f"Found: {list(df.columns)}"
    )

# Force types we care about
df[molecule_col] = df[molecule_col].astype(str)

# --- Build transitions for the single row ---
rows, skipped = [], []
r = df.iloc[0]
mol = r[molecule_col]
pol = norm_polarity(r[pol_col] if pol_col in df.columns else None)
z_list  = extract_int_list(r[zlist_col])
mz_list = extract_float_list(r[mzlist_col])

n = min(len(z_list), len(mz_list))
if n == 0:
    skipped.append((ROW_NUM, mol, "empty or unparsable matched_z_list/matched_mz_list"))
else:
    for z, mz in zip(z_list[:n], mz_list[:n]):
        try:
            z_int = int(z)
        except Exception:
            skipped.append((ROW_NUM, mol, f"non-integer charge '{z}' discarded"))
            continue
        if z_int == 0:
            skipped.append((ROW_NUM, mol, "charge=0 discarded"))
            continue
        rows.append({
            "Molecule": mol,
            "Precursor m/z": f"{float(mz):.6f}",
            "Precursor Adduct": adduct_from_charge(z_int, pol),
            "Polarity": pol
        })

transitions = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)

# --- Save CSV for Skyline ---
out_path = Path(OUTPUT_PATH)
out_path.parent.mkdir(parents=True, exist_ok=True)

# Ensure correct column order even if empty
wanted_cols = ["Molecule", "Precursor m/z", "Precursor Adduct", "Polarity"]
for c in wanted_cols:
    if c not in transitions.columns:
        transitions[c] = []
transitions = transitions[wanted_cols]

transitions.to_csv(out_path, index=False)

# --- Minimal console feedback ---
print(f"Processed only row #{ROW_NUM} from: {p.resolve()}")
print(f"Generated transitions: {len(transitions)}")
if skipped:
    print("Skipped entries (up to 10 shown):")
    for tup in skipped[:10]:
        print("  Row:", tup[0], "| Molecule:", tup[1], "| Reason:", tup[2])
print("Saved →", out_path.resolve())


Processed only row #1 from: F:\input.csv
Generated transitions: 17
Saved → F:\transition_list_row0.csv
